Test the impact of errors in window power spectra on the pseudo-Cl as well as correlation functions. Reproduces Fig. 3 in the paper.

In [1]:
from imaster_paper_args import *

In [6]:
test=False #if true, run test case to make sure code runs.

In [7]:
#some skylens args. 

do_pseudo_cl=True
do_xi=False

use_binned_l=False

do_cov=False

nside=1024
use_window=True

bin_cl=True
bin_xi=True

In [8]:
#setup parameters
lmax_cl=nside*2
lmax2=int(lmax_cl/2)#1000
if test:
    lmax_cl=200
    nside=512
lmin_cl=0
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax2 #1000 #lmax_cl-10
Nl_bins=40
l_bins=np.unique(np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins+1)))
# l_bins=get_l_bins(l_min=lmin_cl_Bins,l_max=lmax_cl_Bins,N_bins=Nl_bins,binning_scheme='log',min_modes=500)
lb=0.5*(l_bins[1:]+l_bins[:-1])
Nl_bins=len(lb)
# l=np.unique(np.int64(np.logspace(np.log10(lmin_cl+1),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell
l=l0
window_lmax=lmax_cl_Bins*2

if window_lmax>3*nside:
    print('nside of window too small')

##  Test with 1 redshift bin

In [10]:
#setup tomographic bins
z0=1
shear_zbins=lsst_source_tomo_bins(nbins=1,use_window=use_window,nside=nside,f_sky=f_sky)
galaxy_zbins=shear_zbins

ns0:  27.0
[0.10002    0.10082016 0.10162032 ... 2.99819964 2.9989998  2.99979996] 100 5
source_tomo_bins, zmax 5.0 2.999799959991998 0.5999699939987997
Tracer utils has no tracers
Tracer utils has tracers:  ['galaxy']
pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 273 3072 31.0 10
Window done. Size: 0.0


/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/cov_utils.py:92: RuntimeWarning: invalid value encountered in true_divide
/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/angular_power_spectra.py:23: RuntimeWarning: divide by zero encountered in true_divide
/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/angular_power_spectra.py:115: RuntimeWarning: divide by zero encountered in true_divide


scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary


/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/healpy-1.14.0-py3.8-linux-x86_64.egg/healpy/sphtfunc.py:433: FutureChangeWarning: The order of the input cl's will change in a future release.
Use new=True keyword to start using the new order.
See documentation of healpy.synalm.
  warnings.warn(
/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/healpy-1.14.0-py3.8-linux-x86_64.egg/healpy/sphtfunc.py:822: UserWarning: Sigma is 0.000000 arcmin (0.000000 rad) 
  warnings.warn(
/verafs/home/sukhdeep/.local/lib/python3.8/site-packages/healpy-1.14.0-py3.8-linux-x86_64.egg/healpy/sphtfunc.py:827: UserWarning: -> fwhm is 0.000000 arcmin
  warnings.warn(
../skylens/survey_utils.py:126: RuntimeWarning: invalid value encountered in sqrt
  cl_map_noise=np.sqrt(cl_map)
../skylens/survey_utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  shear_zbins[i]['window_N']=np.sqrt(shear_zbins[i]['window'])


In [12]:
#choose the cross correlation to test here.
corrs=[corr_gg]
corr=corrs[0]

In [13]:
Skylens_kwargs=parse_dict(locals())

In [14]:
#setup skylens object
#use all ell
Skylens_kwargs['use_window']=False
SL0=Skylens(**Skylens_kwargs)
SL0.gather_data()
Skylens_kwargs['use_window']=True

Tracer utils has tracers:  ['shear', 'galaxy']
pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 9 2048 2049.0 2
Window done. Size: 0.0
gather_dict got empty dictionary
gather_dict got empty dictionary
gather_dict got empty dictionary
gather_dict got empty dictionary


In [15]:
%time cl0G=SL0.cl_tomo()  #compute graph for C_ell

scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary
CPU times: user 26.7 s, sys: 1.36 s, total: 28.1 s
Wall time: 2.11 s


In [16]:
win_Cl=client.compute(cl0G['cl'][corr][bi]).result() #we use same C_ell correlation as window C_l to compute M^W
cl0=win_Cl*1.

In [18]:
#read in wigner-3j matrices.
window_l_test=SL0.l
wu=window_utils(l=l0[:lmax2],window_l=window_l_test,use_window=True,s1_s2s={corr:s1_s2s[corr]},
                corrs=corrs,wigner_step=l0.max()+1,wigner_files=wigner_files)
wu.set_wig3j()
wj2=client.compute(wu.wig_3j_2).result()
wj2=wj2[0]['00']

Win gen: step size 2048 1024 2048.0 10
wigner_files: {0: '/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/temp/dask_wig3j_l2200_w4400_0_reorder.zarr', 2: '/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/temp//dask_wig3j_l2200_w4400_2_reorder.zarr'}
wigner done dict_keys([])


distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 99% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ('set_wig3j_step_multiplied-8703fe89-34e7-49c3-9875-656ffd587a7d', <Worker 'tcp://127.0.0.1:36015', name: 0, memory: 0, processing: 1>)

In [ ]:
win_Cl.shape,wj2.shape

In [ ]:
#compute the coupling matrix
M=wj2@(win_Cl*(2*l0+1))
M/=4.*np.pi
M*=wu.MF



In [ ]:
win_Cl2=win_Cl[:lmax2]*f_sky

In [ ]:
#normailizing coupling matrix for some visual tests
NM=M/np.sqrt(np.outer(win_Cl2,win_Cl2)) 
EM=1./win_Cl2*M
EM2=M*1./win_Cl2

In [ ]:
plot(l0[:lmax2]+1,np.diag(M)) #
xscale('log')
yscale('log')
show()
plot(l0[:lmax2]+1,M@np.ones_like(l0[:lmax2]))
xscale('log')
yscale('log')
xlim(1,1000)
show()

In [ ]:
plot(l0[:lmax2]+1,np.diag(EM)) #
xscale('log')
yscale('log')
show()
plot(l0[:lmax2]+1,EM@np.ones_like(l0[:lmax2])) #
xscale('log')
yscale('log')
show()
plot(l0[:lmax2]+1,np.diag(EM)/(2*l0[:lmax2]+1))
xscale('log')
# yscale('log')
show()
plot(l0[:lmax2]+1,(EM@np.ones_like(l0[:lmax2]))/(2*l0[:lmax2]+1)) #
xscale('log')
yscale('log')
show()


In [ ]:
shear_zbins[0].keys()

In [ ]:
w_alm=hp.map2alm(shear_zbins[0]['window'])
wlt0=hp.anafast(shear_zbins[0]['window'])

In [ ]:
#set up new contaminated window.
fsky=0.3
lmm=3*nside
lt=np.arange(lmm)
Nl=np.mean(wlt0[100:200])
Nl*=2
cov=2./(2*lt+1)/fsky*((wlt0+Nl)**2)
err=np.sqrt(cov)
err[:10]=0
# fl=1+0.4/np.sqrt(lmm)*np.sqrt(lt)
fl=1+np.sqrt(err/wlt0)
# w_alm2=hp.almxfl(w_alm,fl,inplace=False)
w_alm2=w_alm+hp.synalm(err)

In [ ]:
mask=shear_zbins[0]['window']!=hp.UNSEEN
# ww=hp.alm2map(w_alm2,nside=nside)#skylens_kwargs[0]['shear_zbins'][0]['window']*.995
ww=hp.synfast(err,nside=nside)
ww-=ww[mask].mean()
ww[~mask]=hp.UNSEEN
cl_map=ww+shear_zbins[0]['window']
cl_map[~mask]=hp.UNSEEN

cl_mapg=ww+shear_zbins[0]['window']
cl_mapg[~mask]=hp.UNSEEN

wlt=hp.anafast(cl_map)
ww[mask].mean(),cl_map[mask].mean(),shear_zbins[0]['window'][mask].mean()

In [ ]:
shear_zbins2=copy.deepcopy(shear_zbins)
shear_zbins2[0]['window']=cl_map

In [ ]:
Skylens_kwargs['use_window']=True
Skylens_kwargs['use_binned_l']=True
Skylens_kwargs['wigner_step']=50
Skylens_kwargs['l']=l0[:lmax2]

Skylens_kwargs2=copy.deepcopy(Skylens_kwargs)
Skylens_kwargs2['shear_zbins']=shear_zbins2
Skylens_kwargs2['galaxy_zbins']=shear_zbins2

In [ ]:
M0=wj2@(wlt0[:l0.max()+1]*(2*l0+1))
M0/=4.*np.pi
M0*=wu.MF

M2=wj2@(wlt[:l0.max()+1]*(2*l0+1))
M2/=4.*np.pi
M2*=wu.MF


In [ ]:
pcl0=np.zeros_like(SL0.l,dtype='float64')
pcl2=np.zeros_like(SL0.l,dtype='float64')

pcl0[:lmax2]+=M0@cl0[:lmax2]
pcl2[:lmax2]+=M2@cl0[:lmax2]

pcl0_b=SL0.bin_cl_func(cl=pcl0)
pcl2_b=SL0.bin_cl_func(cl=pcl2)

In [ ]:
pcl0

In [ ]:
plot(l0,pcl0)
plot(l0,pcl2)
plot(l0,cl0)
loglog()
show()
plot(l0,1-pcl0/pcl2)
xscale('log')
yscale('log')

In [ ]:
#use all ell
kappa_win={0:Skylens(**Skylens_kwargs)}
kappa_win[0].gather_data()
kappa_win[1]=Skylens(**Skylens_kwargs2)
kappa_win[1].gather_data()

In [ ]:
plot(wlt/wlt0-1)
xscale('log')
yscale('log')
show()
plot(np.absolute(wlt-wlt0))
xscale('log')
yscale('log')

In [ ]:
clG_win={}
cl0_win={}
for i in kappa_win.keys():
    %time clG_win[i]=kappa_win[i].cl_tomo(corrs=corrs) 
    %time cl0_win[i]=clG_win[i]['stack'].compute()
pcl0_b=cl0_win[0]['pcl_b']
pcl2_b=cl0_win[1]['pcl_b']

In [ ]:
SL0.l

In [ ]:
# kappa_win[1].gather_data()
wltb=SL0.bin_cl_func(cl=wlt[SL0.l])
wlt0b=SL0.bin_cl_func(cl=wlt0[SL0.l])

In [ ]:
plot(lb,1.-pcl0_b/pcl2_b)
plot(lb,(wltb/wlt0b-1))
xscale('log')
# plot(lb,cl0_win['t']['pcl_b'])
xscale('log')
yscale('log')
xlabel(r'$\ell$')
ylabel(r'$\frac{D_\ell}{D_{\ell,m}}-1$')
xlim(20,1000)
ylim(0.001,.5)

In [ ]:
plot(lb,(wltb/pcl0_b))
xscale('log')
# plot(lb,cl0_win['t']['pcl_b'])
xscale('log')
yscale('log')
xlabel(r'$\ell$')
ylabel(r'$\frac{D_\ell}{D_{\ell,m}}-1$')
xlim(20,1000)
# ylim(0.001,.5)

In [ ]:
fig, ax =subplots(2,1,figsize=(20,15),sharex='col',sharey='row')
ps=ax[0]
pr=ax[1]
ps.plot(wlt,alpha=0.2)
ps.plot(wlt0,alpha=0.2)
ps.plot(lb,wltb,label='$W_1+\delta W_1$')
ps.plot(lb,wlt0b,label='$W_1$')

ps.plot(lb,pcl0_b,label='$D_\ell, W_1$')
ps.plot(lb,pcl2_b,label='$D_\ell, W_1+\delta W_1$')

pr.plot(lb,1.-pcl0_b/pcl2_b,label=r'$\frac{\delta D_\ell}{D_\ell}$')
pr.plot(lb,(wltb/wlt0b-1),label=r'$\frac{\delta W_1}{W_1}$')

pr.set_ylim(0.001,.5)
pr.set_xlim(1,2000)
pr.hlines(0.01,1,2000,ls='--',color='k')
ps.legend()
pr.legend()
ps.set_yscale('log')
ps.set_xscale('log')
pr.set_yscale('log')
pr.set_xlabel('$\ell$')
ps.set_ylabel('$D_\ell$')
pr.set_ylabel('Fractional difference')
fname=fig_home+'window_bias_Dl'
fname+='.pdf'
show()
# fig.savefig(fname,bbox_inches='tight',format='pdf')

# correlation function

In [ ]:
lmax_xi=10000
l0th=np.arange(lmax_xi)
theta_min=1/60
theta_max=600./60
n_theta_bins=51
theta_bins=np.logspace(np.log10(theta_min),np.log10(theta_max),n_theta_bins+1)
th=np.logspace(np.log10(theta_min),np.log10(theta_max),n_theta_bins*80)

n_theta_bins=21
theta_bins=np.logspace(np.log10(theta_min),np.log10(theta_max),n_theta_bins+1)

theta_bins_center=0.5*(theta_bins[1:]+theta_bins[:-1])
thb=theta_bins_center*1
n_theta_bins=len(thb)
binn=binning()
bu=binn.bin_utils(r=th,r_bins=theta_bins)

In [ ]:
#Hankel Transform setup
WT_kwargs={'l':l0th,'l_cut_weights':np.ones_like(l0th),'theta':th*d2r,'s1_s2':[(0,0)],
           'wig_d_taper_order_low':16,'wig_d_taper_order_high':20}
# if globals().get('WT') is None:
WT=wigner_transform(**WT_kwargs )
# WT_b=wigner_transform(**WT_kwargs)
WT_b=WT
WT.gather_data()

In [ ]:
nside=2048*2

In [ ]:
z0=1 #1087
# zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=use_window,nside=nside)
shear_zbins=lsst_source_tomo_bins(nbins=1,use_window=use_window,nside=nside,f_sky=f_sky)
galaxy_zbins=shear_zbins

In [ ]:
w_alm=hp.map2alm(shear_zbins[0]['window'])
wlt0=hp.anafast(shear_zbins[0]['window'])

In [ ]:
lmm=3*nside
lt=np.arange(lmm)
Nl=np.mean(wlt0[100:200])
Nl*=2
cov=2./(2*lt+1)/fsky*((wlt0+Nl)**2)
err=np.sqrt(cov)
err[:10]=0
# fl=1+0.4/np.sqrt(lmm)*np.sqrt(lt)
# fl=1+np.sqrt(err/wlt0)
# # w_alm2=hp.almxfl(w_alm,fl,inplace=False)
# w_alm2=w_alm+hp.synalm(err)

In [ ]:
mask=shear_zbins[0]['window']!=hp.UNSEEN
# ww=hp.alm2map(w_alm2,nside=nside)#skylens_kwargs[0]['shear_zbins'][0]['window']*.995
ww=hp.synfast(err,nside=nside)
ww-=ww[mask].mean()
ww[~mask]=hp.UNSEEN
cl_map=ww+shear_zbins[0]['window']
cl_map[~mask]=hp.UNSEEN

cl_mapg=ww+shear_zbins[0]['window']
cl_mapg[~mask]=hp.UNSEEN

wlt=hp.anafast(cl_map)
ww[mask].mean(),cl_map[mask].mean(),shear_zbins[0]['window'][mask].mean()

In [ ]:
lw=np.arange(len(wlt0))
th_t,xi0=WT.projected_correlation(l_cl=lw,cl=wlt0,s1_s2=(0,0))
th_t,xi_err=WT.projected_correlation(l_cl=lw,cl=wlt,s1_s2=(0,0))

xi0b=binn.bin_1d(xi=xi0,bin_utils=bu)
xi_errb=binn.bin_1d(xi=xi_err,bin_utils=bu)

In [ ]:
plot(thb,xi0b)
plot(thb,xi_errb)
xscale('log')
show()
plot(thb,xi_errb/xi0b-1)
xscale('log')
yscale('log')